In [ ]:
#Approach 1 Edge detection (Canny Algorithm) And Mean shift Filter
from google.colab.patches import cv2_imshow
import cv2
import numpy as np
import os
import json

def auto_canny(image, sigma=0.80):
    v = np.median(image)
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)
    return edged

ground_truth_file = '/content/drive/MyDrive/Colab Notebooks/MineApple/detection/mapping.json'
folder_path = '/content/drive/MyDrive/Colab Notebooks/MineApple/detection/images'
total_apples = 0
true_positives = 0
false_positives = 0

with open(ground_truth_file, 'r') as f:
    ground_truth = json.load(f)

for filename in os.listdir(folder_path):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Read the image
        image = cv2.imread(os.path.join(folder_path, filename))
        image_d = image.copy()

        # Apply image processing operations
        img_blur = cv2.GaussianBlur(image, (5, 5), 0)
        img_ms = cv2.pyrMeanShiftFiltering(img_blur, 10, 90)
        edge = auto_canny(img_ms)

        # Find contours and draw circles
        cnts, _ = cv2.findContours(edge.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        c_num = 0
        for i, c in enumerate(cnts):
            ((x, y), r) = cv2.minEnclosingCircle(c)
            if r > 10:
                c_num += 1
                cv2.circle(image_d, (int(x), int(y)), int(r), (0, 255, 0), 2)
                cv2.putText(image_d, "#{}".format(c_num), (int(x) - 10, int(y)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
                # Check if the detected apple overlaps with the ground truth apple
                if filename in ground_truth:
                    if c_num <= ground_truth[filename]:
                        true_positives += 1
                    else:
                        false_positives += 1
                else:
                    false_positives += 1

        total_apples += c_num

        # Display the count
        # print("Number of Apples:", c_num)

        # Display the images
        # cv2_imshow(image)
        cv2_imshow(image_d)

        # cv2.waitKey(0)

# Calculate false negatives
false_negatives = total_apples - true_positives

# Calculate precision, recall, and F1-score
precision = true_positives / (true_positives + false_positives)
recall = true_positives / (true_positives + false_negatives)
f1_score = 2 * (precision * recall) / (precision + recall)

print("Total Apples:", total_apples)
print("True Positives:", true_positives)
print("False Positives:", false_positives)
print("False Negatives:", false_negatives)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1_score)

In [ ]:
#Approach 2: Usning color masking. and morphology
import numpy as np
import cv2
import os
from google.colab.patches import cv2_imshow

low_apple_red = (160.0, 153.0, 153.0)
high_apple_red = (180.0, 255.0, 255.0)
low_apple_green = (35.0, 50.0, 50.0)
high_apple_green = (80.0, 255.0, 255.0)
low_apple_raw = (0.0, 150.0, 150.0)
high_apple_raw = (15.0, 255.0, 255.0)
low_apple_golden = (20.0, 100.0, 100.0)
high_apple_golden = (35.0, 255.0, 255.0)
low_apple_over_ripened = (0.0, 150.0, 150.0)
high_apple_over_ripened = (15.0, 255.0, 255.0)

folder_path = '/content/drive/MyDrive/Colab Notebooks/TestData'
ground_truth_file = '/content/drive/MyDrive/Colab Notebooks/TestData/TestDataJson.json'
total_apples = 0
correct_detections = 0
total_apples = 0
true_positives = 0
false_positives = 0
with open(ground_truth_file, 'r') as f:
    ground_truth = json.load(f)

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path) and (filename.endswith('.jpg') or filename.endswith('.png')):
        image_bgr = cv2.imread(file_path)
        image_hsv = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2HSV)

        mask_red = cv2.inRange(image_hsv, low_apple_red, high_apple_red)
        mask_raw = cv2.inRange(image_hsv, low_apple_raw, high_apple_raw)
        mask_green = cv2.inRange(image_hsv, low_apple_green, high_apple_green)
        mask_golden = cv2.inRange(image_hsv,low_apple_golden,low_apple_golden)
        mask_over_ripened=cv2.inRange(image_hsv,low_apple_over_ripened,high_apple_over_ripened)
        mask = mask_red + mask_green + mask_raw + mask_golden +  mask_over_ripened

        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

        _, labels, stats, _ = cv2.connectedComponentsWithStats(mask)

        c_num = 0
        for label in range(1, np.max(labels)+1):
            area = stats[label, cv2.CC_STAT_AREA]
            if area > 100:
                c_num += 1
                centroid_x = stats[label, cv2.CC_STAT_LEFT] + stats[label, cv2.CC_STAT_WIDTH] // 2
                centroid_y = stats[label, cv2.CC_STAT_TOP] + stats[label, cv2.CC_STAT_HEIGHT] // 2

                cv2.circle(image_bgr, (centroid_x, centroid_y), 10, (0, 255, 0), 2)
                cv2.putText(image_bgr, "#{}".format(c_num), (centroid_x - 10, centroid_y),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
                 # Check if the detected apple overlaps with the ground truth apple
                if filename in ground_truth and int(ground_truth[filename]) > 0:
                    correct_detections += 1
                     # Check if the detected apple overlaps with the ground truth apple
                # Check if the detected apple overlaps with the ground truth apple
                if filename in ground_truth:
                    if c_num <= int(ground_truth[filename]):
                        true_positives += 1
                    else:
                        false_positives += 1
                else:
                    false_positives += 1

              # accuracy = correct_detections / c_num
        # print("Accuracy:", accuracy)
        total_apples += c_num
        # Display the count
        print("Number of Apples:", c_num)
        # Display the images
        #cv2_imshow(image)
        # cv2_imshow(img_blur)
        # cv2_imshow(img_ms)
        #cv2_imshow(edge)
        # cv2_imshow(image_d)
        cv2.waitKey(0)
accuracy = correct_detections / total_apples
print("Dataset Accuracy:", accuracy)
# Calculate false negatives
false_negatives = total_apples - true_positives

# Calculate precision, recall, and F1-score
precision = true_positives / (true_positives + false_positives)
recall = true_positives / (true_positives + false_negatives)
f1_score = 2 * (precision * recall) / (precision + recall)

print("Total Apples:", total_apples)
print("True Positives:", true_positives)
print("False Positives:", false_positives)
print("False Negatives:", false_negatives)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1_score)
